# Core Model Guide: PIDController

This notebook provides a detailed guide to the `PIDController`, a fundamental feedback controller used extensively in industrial control systems. It is one of the core control models in the CHS SDK.

## 1. Theoretical Background

A PID (Proportional-Integral-Derivative) controller continuously calculates an **error value** `e(t)` as the difference between a desired **setpoint (SP)** and a measured **process variable (PV)**. It applies a correction based on three terms:

- **Proportional (P)**: Proportional to the current error. A larger P term results in a stronger response to the current error. `P_out = Kp * e(t)`
- **Integral (I)**: Proportional to the accumulation of past errors. The I term eliminates steady-state error by integrating the error over time. `I_out = Ki * ∫e(t)dt`
- **Derivative (D)**: Proportional to the rate of change of the error. The D term predicts future error and can dampen the system's response, reducing overshoot. `D_out = Kd * de(t)/dt`

The final controller output is the sum of these three terms. Tuning the gains `Kp`, `Ki`, and `Kd` is the key to achieving a fast, stable response.

## 2. API and Parameters

Let's import the necessary classes.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.control.pid_controller import PIDController
from water_system_sdk.src.chs_sdk.modules.modeling.storage_models import LinearTank

The `PIDController` is initialized with:

- `Kp` (float): The proportional gain.
- `Ki` (float): The integral gain.
- `Kd` (float): The derivative gain.
- `set_point` (float): The target value for the process variable.
- `windup_guard` (float, optional): An anti-windup limit for the integral term.

## 3. Code Example: Controlling a Tank's Water Level

We will create a simulation to control the water level of a `LinearTank`. The `PIDController` will adjust the `inflow` to the tank to try and maintain the water level at a `set_point` of 10 meters.

In [ ]:
# 1. Model Initialization
tank = LinearTank(area=1000.0, initial_level=2.0)

# Initialize three different PID controllers to compare their performance
pid_well_tuned = PIDController(Kp=2.0, Ki=0.1, Kd=0.5, set_point=10.0)
pid_aggressive = PIDController(Kp=5.0, Ki=0.5, Kd=0.1, set_point=10.0) # High Kp and Ki
pid_oscillating = PIDController(Kp=10.0, Ki=1.5, Kd=0.01, set_point=10.0) # Very high gains

controllers = {
    "Well-Tuned": pid_well_tuned,
    "Aggressive (Overshoot)": pid_aggressive,
    "Oscillating": pid_oscillating
}

# 2. Simulation Setup
dt = 1.0
n_steps = 200
all_history = {}

# 3. Simulation Loop for each controller
for name, controller in controllers.items():
    # Reset the tank for each simulation
    tank = LinearTank(area=1000.0, initial_level=2.0)
    history = []
    
    for t in range(n_steps):
        # The controller's feedback signal is the current tank level
        feedback_signal = tank.level
        
        # The controller calculates the required inflow
        control_output = controller.step(dt=dt, feedback_signal=feedback_signal)
        
        # Set the tank's inflow to the controller's output
        # Ensure inflow is non-negative
        tank.input.inflow = max(0, control_output)
        
        # Simulate the tank for one step
        tank.step(dt=dt)
        
        history.append(tank.level)
        
    all_history[name] = history

print("All simulations complete.")

## 4. Visualization

Plotting the results from the three different controllers clearly shows the effect of tuning the PID gains.

In [ ]:
plt.figure(figsize=(12, 7))

time_axis = np.arange(0, n_steps * dt, dt)

for name, history in all_history.items():
    plt.plot(time_axis, history, label=name)

plt.axhline(y=10.0, color='r', linestyle='--', label='Setpoint (10.0m)')
plt.title('Comparison of PID Controller Tuning')
plt.xlabel('Time (s)')
plt.ylabel('Water Level (m)')
plt.legend()
plt.grid(True)
plt.show()

### Analysis of Results

- **Well-Tuned**: This controller provides a smooth, quick rise to the setpoint with minimal overshoot.
- **Aggressive (Overshoot)**: The higher gains cause the controller to react more strongly, leading to the water level overshooting the setpoint before settling down.
- **Oscillating**: With very high gains, the controller overcorrects so much that it creates sustained oscillations around the setpoint, never fully settling. This demonstrates an unstable control system.

This notebook shows how the `PIDController` can be used to create a closed-loop control system and illustrates the importance of proper gain tuning.